# 認識一下這些資料 ~ Penguin

In [1]:
from pathlib import Path
import json 
data_dir=r"c:\Users\User\Desktop\my_ML\2021\HW4\Dataset"
metadata_path = Path(data_dir) / "metadata.json"
metadata = json.load(open(metadata_path))["speakers"]

In [2]:
Path(data_dir)   
# 答:  WindowsPath('c:/Users/User/Desktop/my_ML/2021/HW4/Dataset')
# 去研究 Path 跟 WindowsPath 類應該又歪掉了
# 反正就是弄出網址的手段，我都用 import os , os.path.其他東西  

metadata0=json.load(open(metadata_path))
(   len(metadata0),list(metadata0)[0],metadata0['n_mels'],list(metadata0)[1],len(metadata)   ), list(metadata)[0:10]

((2, 'n_mels', 40, 'speakers', 600),
 ['id10473',
  'id10328',
  'id10382',
  'id11088',
  'id10553',
  'id11063',
  'id10358',
  'id10218',
  'id10368',
  'id10209'])

In [3]:
metadata[      list(metadata)[0]    ]

[{'feature_path': 'uttr-5c88b2f1803449789c36f14fb4d3c1eb.pt', 'mel_len': 652},
 {'feature_path': 'uttr-022a67baccc54bfda3567a7ac282a7b8.pt', 'mel_len': 564},
 {'feature_path': 'uttr-6a5c6e7231d642568633db13b6e429e1.pt', 'mel_len': 952},
 {'feature_path': 'uttr-e316c0cfdc8843c3ba3cfbb86af687b8.pt', 'mel_len': 1214},
 {'feature_path': 'uttr-3060b2b96bfc4614bb518914e557fea6.pt', 'mel_len': 353},
 {'feature_path': 'uttr-f1ac81adf7ba44c691ba389c9d171a05.pt', 'mel_len': 864},
 {'feature_path': 'uttr-930c993be52a4843b27f4961a44ecfe9.pt', 'mel_len': 352},
 {'feature_path': 'uttr-dbf99fe8f6b64cbb99e868e298813a4a.pt', 'mel_len': 429},
 {'feature_path': 'uttr-788cb6b47da94b34af1afb7e3a10f3f7.pt', 'mel_len': 276},
 {'feature_path': 'uttr-f0ff720eaad94ca9a383518c8e2fef59.pt', 'mel_len': 1560},
 {'feature_path': 'uttr-fcba6892138544d9ae2b199e709d327c.pt', 'mel_len': 594},
 {'feature_path': 'uttr-87460fee9ab44adba2b8b20c2f735686.pt', 'mel_len': 1352},
 {'feature_path': 'uttr-52bfb12f2ace4aa98f9799d15

一樣的要建立 myDataset , __len__, __getitem__ 

.key() 是什麼意思？ 是dict_key 物件， 跟 list(  metadata) 很像

mel 是什麼？ 把 mel 理解為一段聲音的數字記錄

json 檔案格式是什麼？ 對了，看一下助教影片吧！它就是記錄 dict 的東西，只要 import json 就能讀進來變成 dict

助教講的 d_model 不知道是幹嘛的，是那個隨機 sample 舉例的 2 嗎？  .pt 檔裡面到底裝什麼？ 

In [4]:
speaker=list(metadata.keys())[0] 
metadata.keys()
type(metadata) 

dict

In [11]:
speaker=list(metadata.keys())[0]     
#  第一名講者
#  奇怪的資料型態  type(metadata.keys())==list  --> False 

mapping_path=r"c:\Users\User\Desktop\my_ML\2021\HW4\Dataset\mapping.json"
mapping = json.load(open(mapping_path))
speaker2id = mapping["speaker2id"]
mapping.keys() 
# speaker2id  # {'id10001': 0, 'id10005': 1,   'id10006': 2,  'id10007': 3,  'id10008': 4, .... }
# mapping['id2speaker']  # '0': 'id10001', '1': 'id10005', '2': 'id10006', '3': 'id10007', '4': 'id10008',...}

dict_keys(['speaker2id', 'id2speaker'])

In [12]:

data=[]
for utterances in metadata[speaker]:
    data.append([utterances["feature_path"], speaker2id[speaker]])
data[0:5]

 

[['uttr-5c88b2f1803449789c36f14fb4d3c1eb.pt', 217],
 ['uttr-022a67baccc54bfda3567a7ac282a7b8.pt', 217],
 ['uttr-6a5c6e7231d642568633db13b6e429e1.pt', 217],
 ['uttr-e316c0cfdc8843c3ba3cfbb86af687b8.pt', 217],
 ['uttr-3060b2b96bfc4614bb518914e557fea6.pt', 217]]

到目前為止 data  就是針對某一個講者的所有錄音紀錄，含有他的編號(id)，如上。 在底下每一個 .pt 檔讀出來都是一個向量的序列，會看到 shape 是 [652,40] ，代表是 652 項的 $\mathbb R^{40}$ 的元素。 其中的 652 理解為上面的 mel_len ：

In [15]:
index = 0 
feat_path, speaker = data[index]


import torch 
import os 
mel = torch.load(os.path.join(data_dir, feat_path))
mel.shape
#len(mel)

# ------------
# 執行比較久, 4.2秒 
# ------------

torch.Size([652, 40])

In [8]:
type(mel)   #torch.Tensor
mel = torch.FloatTensor(mel)
type(mel)  



torch.Tensor

把長度超過 segment_len=128 的截取其中的 128 的一段出來： 

In [9]:


import random
segment_len=128
if len(mel) > segment_len:
  # Randomly get the starting point of the segment.
  start = random.randint(0, len(mel) - segment_len)
  # Get a segment with "segment_len" frames.
  mel = torch.FloatTensor(mel[start:start+segment_len])
else:
  mel = torch.FloatTensor(mel)
# Turn the speaker id into long for computing loss later.
speaker = torch.FloatTensor([speaker]).long()


mel.shape, speaker, len(mel) 


(torch.Size([128, 40]), tensor([217]), 128)

以上截取出了一組訓練資料 $(\vec{x},y)$=(mel,speaker)。底下取出一個批次： 

In [18]:
import torch 
import os 
import random


batch=[] 
batch_size=5
segment_len=128

for index in range(batch_size):
    feat_path, speaker = data[index]
    mel = torch.load(os.path.join(data_dir, feat_path))
    
    if len(mel) > segment_len:
    # Randomly get the starting point of the segment.
        start = random.randint(0, len(mel) - segment_len)
    # Get a segment with "segment_len" frames.
        mel = torch.FloatTensor(mel[start:start+segment_len])
    else:
        mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()

    print(mel.shape)
    batch.append((mel,speaker))

len(batch), type(batch[0]),len(batch[0]),batch[0][0].shape

torch.Size([128, 40])
torch.Size([128, 40])
torch.Size([128, 40])
torch.Size([128, 40])
torch.Size([128, 40])


(5, tuple, 2, torch.Size([128, 40]))

## DataLoader 的問題

### collate_fn

在底下的 DataLoader 的部份多了 collate_fn 這個項要寫，即 

        DataLoader(     
            trainset,    
            batch_size=batch_size,    
            collate_fn=collate_batch
            )
  
所以先來理解與它相關的種種。 

從[pytorch之深入理解collate_fn](https://blog.csdn.net/qq_43391414/article/details/120462055) 知道，DataLoader 會先讀進 batch_size 這麼多筆資料，形成一個批次，接著把這個批次送進 collate_fn 函式再處理，才成為真正的批次。 如果 collate_fn 沒特別指定，那就是torch 內定的函式。 

而這裡的 collate_fn 我們指定為 collate_batch。  

(這是看了看就忘了的參考資料(collate_fn)： https://zhuanlan.zhihu.com/p/346332974 ) 


## collate_batch 裡的問題

In [20]:
def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()

In [22]:
mel, speaker = zip(*batch)
mel, speaker 

# zip(*batch) 看不出來 zip 在哪裡
# 實在是看不出 pad_sequence 的效果。 

((tensor([[ 3.7789,  5.0095,  6.2048,  ..., -1.7800, -2.6364, -4.1402],
          [ 3.1319,  4.0267,  5.5865,  ..., -2.6058, -3.0292, -3.5199],
          [ 3.5766,  3.6933,  5.1692,  ..., -1.9463, -2.5324, -3.1807],
          ...,
          [ 3.5081,  1.7898,  1.3294,  ..., -3.6261, -4.8100, -6.2782],
          [ 2.3507,  1.5578, -0.2451,  ..., -4.6477, -4.9963, -6.1467],
          [ 2.8042,  1.7832,  0.5915,  ..., -4.9164, -5.5058, -6.6358]]),
  tensor([[ 1.2233,  0.7965, -0.3329,  ..., -6.3155, -6.4495, -6.1057],
          [-0.4542, -0.6395, -1.0690,  ..., -5.9986, -5.3997, -5.7342],
          [-1.3774, -1.0288, -0.4304,  ..., -5.8196, -5.7786, -6.4971],
          ...,
          [-0.1968, -0.0100, -0.8153,  ..., -2.9740, -3.0045, -2.9300],
          [-0.7895,  0.5788,  0.4039,  ..., -3.2665, -2.4843, -2.6286],
          [ 1.5114,  2.0339,  1.9580,  ..., -2.4905, -2.5871, -2.4693]]),
  tensor([[-1.1504, -0.4570,  1.2115,  ..., -4.6025, -5.6709, -6.5668],
          [ 0.2562,  0.5291,  

所以需要做點實測。 

#### padding = -20 的部份

https://blog.csdn.net/m0_37586991/article/details/89467955 https://blog.csdn.net/HUSTHY/article/details/123916820


#### zip(*batch) 的部份: 

(ref:  zip(*batch) https://blog.csdn.net/kz_java/article/details/124650719 https://ithelp.ithome.com.tw/articles/10218029)



底下隨意用個 $4\times 3$ 的矩陣造出資料集，當然造了一個 mytest 資料集進行測試，collate_fn 設成 my_fn，定義為 identity。 如此一來便試試看 zip 會得到什麼東西： 



In [25]:
import torch ,numpy 
from torch.utils.data import Dataset ,DataLoader 
from torch.nn.utils.rnn import pad_sequence

class mytest(Dataset):
    def __init__(self,data):
        self.data=data
    def __getitem__(self,idx):
        return self.data[idx,0:2], self.data[idx,2]
    def __len__(self):
        return len(self.data)
    
a=numpy.random.random((4,3))
print(a,end='\n\n') 
testdata=mytest(a)


def my_fn(x):
    y=[]
    for i in x: 
        i=list(i)
        i[1]+=100
        i=tuple(i) 
        y.append(i) 
    return y 

把a複製出來而已='''  
[ ([0.23314527, 0.05247574], 0.63437053) ,
  ([0.45916101, 0.2706865],  0.4225151) , 
  ([0.25938324, 0.75807604], 0.78212523) ,
  ([0.42055378, 0.80188126], 0.95242862 ),]
  https://pythontutor.com/render.html#mode=display
'''

可在Python_Visualization試試_一個程失敗的寫法_弄清list跟tupe變數的指標本質='''  
x=[ ([0.23314527, 0.05247574], 0.63437053) ,
  ([0.45916101, 0.2706865],  0.4225151) , 
]
  
def my_fn(x):
    for i in x: 
        i=list(i)
        i[1]+=100
        i=tuple(i) 
    return x
    
c=my_fn(x)
'''

dataloader = DataLoader(testdata, batch_size=2, collate_fn=my_fn)
batch=list(dataloader)[0]

mel, speaker = zip(*batch)
# 這邊的 mel 是很多個 mel , speaker 是很多個 speaker
# zip 有拉鏈、拉開的意思 
# 拉鏈拉上時是左跟右，一對一對綁在一起
# 拉鏈拉下後是左邊自己一排，右邊自己一排。 
# *batch 的星號則應該是 python 函數裡的 function(變數, *列表或序對, **字典) 格式的第二節
print(batch)
print( mel, speaker) 

mel=torch.FloatTensor(mel)
mel = pad_sequence(mel, batch_first=True, padding_value=-20) 
mel 

# 執行兩次 warning 會消失


[[0.71511142 0.73934188 0.19947609]
 [0.64381424 0.03567123 0.29782309]
 [0.60832829 0.25268904 0.93861829]
 [0.25464282 0.71424791 0.5423858 ]]

[(array([0.71511142, 0.73934188]), 100.1994760943883), (array([0.64381424, 0.03567123]), 100.29782309383232)]
(array([0.71511142, 0.73934188]), array([0.64381424, 0.03567123])) (100.1994760943883, 100.29782309383232)


tensor([[0.7151, 0.7393],
        [0.6438, 0.0357]])

In [27]:
a=torch.rand(3)
b=torch.rand(5)
c=torch.rand(7)
pad_sequence([a,b,c],batch_first=True,padding_value=-20) 

# 補上 -20 而已， batch_first 差不多是直的排跟橫的排


tensor([[  0.1020,   0.0501,   0.3898, -20.0000, -20.0000, -20.0000, -20.0000],
        [  0.4938,   0.1357,   0.7109,   0.7387,   0.0449, -20.0000, -20.0000],
        [  0.7572,   0.7689,   0.2584,   0.9882,   0.8434,   0.1651,   0.3293]])

如果有把652截取成128，那麼這一步 padding_value 就沒有用途了。 所以底下來查一下到底有沒有哪一段長度 < 128 ： 

#### 為了理解再下一格裡 for i in metadata 的意思

In [42]:
it=iter(metadata)
it   # <dict_keyiterator at 0x1b76634ae50>

i=next(it)   # i = 'id10473'
#list(metadata)[0]  # 'id10473'

it2=iter(metadata[i] ) 
j=it2.__next__()   # 等價於 j=next(it2)
j['mel_len']>=128  

True

#### 尋找長度 < 150 的 


In [28]:
'''  
找出長度小於指定數的所有 mel 和它的 speaker
'''

for i in metadata: #0['speakers']:
    for j in metadata[i]:
        if j['mel_len']<150 :   # 原是 128
            print(i, j['feature_path'] ,j['mel_len'])

id10986 uttr-c1f39fbeff39408684786b10595439fe.pt 129
id11111 uttr-7d9b6e1c007b41419915a21b3946b626.pt 146
id11111 uttr-c230eb5bd3a0400e900b08b18c265db8.pt 148
id11060 uttr-f71b58229faf4c808b634b2512ed3e42.pt 96
id10220 uttr-d0e4b0c8a4334ba9b6a5c2638550ce9d.pt 129


## 進入模型架構的部份

假設以 [652,40] 的那個 mel 為例，在資料處理的階段已經把它截成 [128,40] 的尺寸。 底下的NN結構是先把 [128,40] 中的每個 $\mathbb R^{40}$ 線性地變成 $\mathbb R^{80}$，再丟進 attention 架構，不論是內積自專注或是並行自專注(multihead)。 

因此 $\rm{d}_{\rm{model}}$ 所指的意思是進入 attention 的向量序列中，每一個向量的維度。 而根據 [原始論文](.....)，公式是

$$\rm{sfmx}\it\left(  \frac{QK^{\rm{T}}}{\sqrt{d_k}} \right)V,$$

不過這裡的向量是列向量，李宏毅老師 [上課](https://www.youtube.com/watch?v=gmsMY5kc-zw#t=14m7s) 慣用的是行向量。假設輸入的向量串是 
$$I = \begin{pmatrix}|&&|\\a_1&\cdots&a_n\\|&&|\end{pmatrix}\in \mathbb R^{  d_{\rm model}      \times \it n}$$ 
因此
$$Q= \begin{pmatrix}|&&|\\q_1&\cdots&q_n\\|&&|\end{pmatrix},\,\,\,K=\begin{pmatrix}|&&|\\k_1&\cdots&k_n\\|&&|\end{pmatrix},\,\,\,V=\begin{pmatrix}|&&|\\v_1&\cdots&v_n\\|&&|\end{pmatrix},$$ 
分別可表示為
$$Q=W^{\rm{q}}I,\,\,K=W^{\rm{k}}I, \,\, V=W^{\rm{v}}I\text{。}$$
這裡的 $W^{\rm q},\it W^{\rm k} \in\mathbb R^{ d_1 \times d_{\rm model \it}}$，而$W^{\rm v}\in\mathbb R^{ d_{\rm output} \times d_{\rm model \it}}$。 下一步計算

$$A=K^{\rm T}Q, \quad \it A' = \rm sfmx \it(A),\quad O =  V A' \text{，}$$ 

此即輸出。 如果我們只關注 $Q,K,V$ 如何變成 $O$，那答案就會是論文裡的轉置： 
$$ V\cdot \rm sfmx(\it K^{\rm T\it}V).$$ 

.

有趣的是，向量的個數 n 一直都沒有在考量的範圍內。自專注就是這麼專注在其他變量上！ 話雖如此，但底下的程式碼根本不甩基本款的 Scalar Product Self-Attention： 



In [29]:
import torch.nn as nn 



class Classifier(nn.Module):
  def __init__(self, d_model=80, n_spks=600, dropout=0.1):
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    self.prenet = nn.Linear(40, d_model)
    # TODO:
    #   Change Transformer to Conformer.
    #   https://arxiv.org/abs/2005.08100
    self.encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model, dim_feedforward=256, nhead=2
    )
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

    # Project the the dimension of features from d_model into speaker nums.
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, d_model),
      nn.ReLU(),
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model)
    out = self.prenet(mels)
    # out: (length, batch size, d_model)
    out = out.permute(1, 0, 2)
    # The encoder layer expect features in the shape of (length, batch size, d_model).
    out = self.encoder_layer(out)
    # out: (batch size, length, d_model)
    out = out.transpose(0, 1)
    # mean pooling
    stats = out.mean(dim=1)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out

speaker_num=600 
device='cpu'
model = Classifier(n_spks=speaker_num).to(device)

list(model.encoder_layer.state_dict()) 

['self_attn.in_proj_weight',
 'self_attn.in_proj_bias',
 'self_attn.out_proj.weight',
 'self_attn.out_proj.bias',
 'linear1.weight',
 'linear1.bias',
 'linear2.weight',
 'linear2.bias',
 'norm1.weight',
 'norm1.bias',
 'norm2.weight',
 'norm2.bias']

# self attention 層

那麼， TransformerEncoderLayer 究竟裝啥？ dim_feedforward 是什麼？ 

In [31]:
 # dim_feedforward

In [33]:
import torch.nn as nn 
d_model = 80 
encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model, dim_feedforward=256, nhead=2
    )

I = torch.rand(128,80) 
I.shape   # torch.Size([128, 80])

dir(encoder_layer)
dir(encoder_layer.self_attn.out_proj) 
encoder_layer.self_attn.out_proj.weight.data.shape 
encoder_layer


TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=80, out_features=80, bias=True)
  )
  (linear1): Linear(in_features=80, out_features=256, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=256, out_features=80, bias=True)
  (norm1): LayerNorm((80,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((80,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

不做不知道，一做嚇一跳。這些 linear, dropout , linear 的是什麼？ 還好幾個.... 這個測試還把一個 $\mathbb R^{128\times 80}$ 的向量序列丟進去，發現得到的輸出也是 $\mathbb R^{128\times 80}$ 。 從概念上合理，但技術上卻看不出與 

        dim_feedforward=256 


的關聯。 

進去看 TransformerEncoderLayer 的程式碼吧。 一些簡要結果紀錄在 

        torch的一堆transformer.ipynb 
        
中， 遇到第一個比較大的困擾是 Dropout 是什麼，因為它沒在 [老師的投影片](https://www.youtube.com/watch?v=n9TlOhRjYoc#t=29m) 裡出現。  





### Dropout 是啥!? 

In [34]:
# https://www.cnblogs.com/luckyplj/p/13424561.html 
import torch 
import torch.nn as nn 
for i in range(2):   # 以下動作做兩次

    a=torch.rand(10,1)   # 隨機的 10 維向量
    b=torch.ones(10,3)   # 待會觀察結果用的
    b[:,0]=a[:,0]
    
    b[:,1]=nn.Dropout(0.5)(a)[:,0]   
    # 對 a 使用剔除機率 = 0.5 的 Dropout 

    b[:,2]=b[:,1]/b[:,0]
    # 觀察保留的數據漲了幾倍

    print('\n', b)  


 tensor([[0.2376, 0.0000, 0.0000],
        [0.3319, 0.0000, 0.0000],
        [0.3795, 0.0000, 0.0000],
        [0.6173, 0.0000, 0.0000],
        [0.8108, 0.0000, 0.0000],
        [0.6467, 1.2934, 2.0000],
        [0.7943, 0.0000, 0.0000],
        [0.9298, 1.8597, 2.0000],
        [0.3702, 0.0000, 0.0000],
        [0.7718, 0.0000, 0.0000]])

 tensor([[0.1940, 0.3880, 2.0000],
        [0.2710, 0.5420, 2.0000],
        [0.9168, 0.0000, 0.0000],
        [0.9338, 0.0000, 0.0000],
        [0.5225, 0.0000, 0.0000],
        [0.3371, 0.0000, 0.0000],
        [0.6093, 0.0000, 0.0000],
        [0.1278, 0.2557, 2.0000],
        [0.0063, 0.0126, 2.0000],
        [0.9213, 0.0000, 0.0000]])


設0.5時，被保留的數字大致上在 $ 5\pm 1$ 居多，$ 5\pm 2$ 遠一點的情況較少，符合分佈。 設 0.1的話大致在 $9\pm 1$ 左右，設 0.9 的話在 $2\pm 2$ 。 

## TransformerEncoderLayer 怎麼做到的

最主要的問題還是，中間的 dim_feedforward = 256 究竟在哪裡發生的。 畢竟就連 Encoder 的結構都知道後，看著各個有可能從 80 維漲到256維的部份主要就兩個地方： 

1. attention 的輸出
2. FC 的中間層

如果答案是1，那為 256維的名字是用 feedforward，覺得不合理。 若答案是 2，但 FC 是包在裡面的，我的直覺判斷是，它不會卡在裡面。 為什麼敢這麼武斷？因為認為 FC 的維度不會只變一次 (HW1, HW3 得到的認知)，再加上它沒有激發函數；另一邊，又因為 256 就出現在上面的 linear1 裡面，它並不像是 FC 裡的樣子。 

所以 linear1 在哪裡就成了關鍵： 

### 回答： 

它的 forward 函式主要部份是這樣寫的，輸入是 x，張量，它做了以下： 

        x = self.norm1(x + self._sa_block(x))
        x = self.norm2(x + self._ff_block(x))

回傳 x。 其中 _sa_block 是

        def _sa_block(self, x: Tensor,
                  attn_mask: Optional[Tensor], key_padding_mask: Optional[Tensor], is_causal: bool = False
                  ) -> Tensor:
                  
                x = self.self_attn(x, x, x,
                           attn_mask=attn_mask,
                           key_padding_mask=key_padding_mask,
                           need_weights=False, is_causal=is_causal)[0]
                return self.dropout1(x)，

而  _ff_block 是

        def _ff_block(self, x: Tensor) -> Tensor:
        
                x = self.linear2(self.dropout(self.activation(self.linear1(x))))
                return self.dropout2(x)。 

因此可以看出在 TransformerEncoderLayer 的輸入到輸出，這個 x 經歷了什麼。 結局是，linear1, linear2 還真的是 FC 裡的東西。 




### 研究蟲蟲

In [1]:
# get_cosine_schedule_with_warmup

# 雜亂無章未旅順 

### 先拿到一個 batch 的資料

In [32]:
batch 

[(array([0.71511142, 0.73934188]), 100.1994760943883),
 (array([0.64381424, 0.03567123]), 100.29782309383232)]

In [31]:
model = Classifier(n_spks=speaker_num).to(device)


batch = next(train_iterator)

criterion = nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=1e-3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mels, labels = batch
mels = mels.to(device)
labels = labels.to(device)

outs = model(mels)

loss = criterion(outs, labels)


NameError: name 'train_iterator' is not defined

In [ ]:
import os
import json
import torch
import random
from pathlib import Path
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
 
 
class myDataset(Dataset):
  def __init__(self, data_dir, segment_len=128):
    self.data_dir = data_dir
    self.segment_len = segment_len
 
    # Load the mapping from speaker neme to their corresponding id. 
    mapping_path = Path(data_dir) / "mapping.json"
    mapping = json.load(mapping_path.open())
    self.speaker2id = mapping["speaker2id"]
 
    # Load metadata of training data.
    metadata_path = Path(data_dir) / "metadata.json"
    metadata = json.load(open(metadata_path))["speakers"]
 
    # Get the total number of speaker.
    self.speaker_num = len(metadata.keys())
    self.data = []
    for speaker in metadata.keys():
      for utterances in metadata[speaker]:
        self.data.append([utterances["feature_path"], self.speaker2id[speaker]])
 
  def __len__(self):
    return len(self.data)
 
  def __getitem__(self, index):
    feat_path, speaker = self.data[index]
    # Load preprocessed mel-spectrogram.
    mel = torch.load(os.path.join(self.data_dir, feat_path))
 
    # Segmemt mel-spectrogram into "segment_len" frames.
    if len(mel) > self.segment_len:
      # Randomly get the starting point of the segment.
      start = random.randint(0, len(mel) - self.segment_len)
      # Get a segment with "segment_len" frames.
      mel = torch.FloatTensor(mel[start:start+self.segment_len])
    else:
      mel = torch.FloatTensor(mel)
    # Turn the speaker id into long for computing loss later.
    speaker = torch.FloatTensor([speaker]).long()
    return mel, speaker
 
  def get_speaker_number(self):
    return self.speaker_num
  





import torch
from torch.utils.data import DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


def collate_batch(batch):
  # Process features within a batch.
  """Collate a batch of data."""
  mel, speaker = zip(*batch)
  # Because we train the model batch by batch, we need to pad the features in the same batch to make their lengths the same.
  mel = pad_sequence(mel, batch_first=True, padding_value=-20)    # pad log 10^(-20) which is very small value.
  # mel: (batch size, length, 40)
  return mel, torch.FloatTensor(speaker).long()


def get_dataloader(data_dir, batch_size, n_workers):
  """Generate dataloader"""
  dataset = myDataset(data_dir)
  speaker_num = dataset.get_speaker_number()
  # Split dataset into training dataset and validation dataset
  trainlen = int(0.9 * len(dataset))
  lengths = [trainlen, len(dataset) - trainlen]
  trainset, validset = random_split(dataset, lengths)

  train_loader = DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=collate_batch,
  )
  valid_loader = DataLoader(
    validset,
    batch_size=batch_size,
    num_workers=n_workers,
    drop_last=True,
    pin_memory=True,
    collate_fn=collate_batch,
  )

  return train_loader, valid_loader, speaker_num


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Classifier(nn.Module):
  def __init__(self, d_model=80, n_spks=600, dropout=0.1):
    super().__init__()
    # Project the dimension of features from that of input into d_model.
    self.prenet = nn.Linear(40, d_model)
    # TODO:
    #   Change Transformer to Conformer.
    #   https://arxiv.org/abs/2005.08100
    self.encoder_layer = nn.TransformerEncoderLayer(
      d_model=d_model, dim_feedforward=256, nhead=2
    )
    # self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=2)

    # Project the the dimension of features from d_model into speaker nums.
    self.pred_layer = nn.Sequential(
      nn.Linear(d_model, d_model),
      nn.ReLU(),
      nn.Linear(d_model, n_spks),
    )

  def forward(self, mels):
    """
    args:
      mels: (batch size, length, 40)
    return:
      out: (batch size, n_spks)
    """
    # out: (batch size, length, d_model)
    out = self.prenet(mels)
    # out: (length, batch size, d_model)
    out = out.permute(1, 0, 2)
    # The encoder layer expect features in the shape of (length, batch size, d_model).
    out = self.encoder_layer(out)
    # out: (batch size, length, d_model)
    out = out.transpose(0, 1)
    # mean pooling
    stats = out.mean(dim=1)

    # out: (batch, n_spks)
    out = self.pred_layer(stats)
    return out


In [4]:
import math

import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_cosine_schedule_with_warmup(
  optimizer: Optimizer,
  num_warmup_steps: int,
  num_training_steps: int,
  num_cycles: float = 0.5,
  last_epoch: int = -1,
):
  """
  Create a schedule with a learning rate that decreases following the values of the cosine function between the
  initial lr set in the optimizer to 0, after a warmup period during which it increases linearly between 0 and the
  initial lr set in the optimizer.

  Args:
    optimizer (:class:`~torch.optim.Optimizer`):
      The optimizer for which to schedule the learning rate.
    num_warmup_steps (:obj:`int`):
      The number of steps for the warmup phase.
    num_training_steps (:obj:`int`):
      The total number of training steps.
    num_cycles (:obj:`float`, `optional`, defaults to 0.5):
      The number of waves in the cosine schedule (the defaults is to just decrease from the max value to 0
      following a half-cosine).
    last_epoch (:obj:`int`, `optional`, defaults to -1):
      The index of the last epoch when resuming training.

  Return:
    :obj:`torch.optim.lr_scheduler.LambdaLR` with the appropriate schedule.
  """

  def lr_lambda(current_step):
    # Warmup
    if current_step < num_warmup_steps:
      return float(current_step) / float(max(1, num_warmup_steps))
    # decadence
    progress = float(current_step - num_warmup_steps) / float(
      max(1, num_training_steps - num_warmup_steps)
    )
    return max(
      0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))
    )

  return LambdaLR(optimizer, lr_lambda, last_epoch)


In [5]:
import torch


def model_fn(batch, model, criterion, device):
  """Forward a batch through the model."""

  mels, labels = batch
  mels = mels.to(device)
  labels = labels.to(device)

  outs = model(mels)

  loss = criterion(outs, labels)

  # Get the speaker id with highest probability.
  preds = outs.argmax(1)
  # Compute accuracy.
  accuracy = torch.mean((preds == labels).float())

  return loss, accuracy


In [6]:
from tqdm import tqdm
import torch


def valid(dataloader, model, criterion, device): 
  """Validate on validation set."""

  model.eval()
  running_loss = 0.0
  running_accuracy = 0.0
  pbar = tqdm(total=len(dataloader.dataset), ncols=0, desc="Valid", unit=" uttr")

  for i, batch in enumerate(dataloader):
    with torch.no_grad():
      loss, accuracy = model_fn(batch, model, criterion, device)
      running_loss += loss.item()
      running_accuracy += accuracy.item()

    pbar.update(dataloader.batch_size)
    pbar.set_postfix(
      loss=f"{running_loss / (i+1):.2f}",
      accuracy=f"{running_accuracy / (i+1):.2f}",
    )

  pbar.close()
  model.train()

  return running_accuracy / len(dataloader)


In [8]:
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, random_split


def parse_args():
  """arguments"""
  config = {
    "data_dir": r"c:\Users\User\Desktop\Dataset",
    "save_path": "model.ckpt",
    "batch_size": 32,
    "n_workers": 8,
    "valid_steps": 2000,
    "warmup_steps": 1000,
    "save_steps": 10000,
    "total_steps": 70000,
  }

  return config


def main(
  data_dir,
  save_path,
  batch_size,
  n_workers,
  valid_steps,
  warmup_steps,
  total_steps,
  save_steps,
):
  """Main function."""
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"[Info]: Use {device} now!")

  train_loader, valid_loader, speaker_num = get_dataloader(data_dir, batch_size, n_workers)
  train_iterator = iter(train_loader)
  print(f"[Info]: Finish loading data!",flush = True)

  model = Classifier(n_spks=speaker_num).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = AdamW(model.parameters(), lr=1e-3)
  scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
  print(f"[Info]: Finish creating model!",flush = True)

  best_accuracy = -1.0
  best_state_dict = None

  pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

  for step in range(total_steps):
    # Get data
    try:
      batch = next(train_iterator)
    except StopIteration:
      train_iterator = iter(train_loader)
      batch = next(train_iterator)

    loss, accuracy = model_fn(batch, model, criterion, device)
    batch_loss = loss.item()
    batch_accuracy = accuracy.item()

    # Updata model
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    
    # Log
    pbar.update()
    pbar.set_postfix(
      loss=f"{batch_loss:.2f}",
      accuracy=f"{batch_accuracy:.2f}",
      step=step + 1,
    )

    # Do validation
    if (step + 1) % valid_steps == 0:
      pbar.close()

      valid_accuracy = valid(valid_loader, model, criterion, device)

      # keep the best model
      if valid_accuracy > best_accuracy:
        best_accuracy = valid_accuracy
        best_state_dict = model.state_dict()

      pbar = tqdm(total=valid_steps, ncols=0, desc="Train", unit=" step")

    # Save the best model so far.
    if (step + 1) % save_steps == 0 and best_state_dict is not None:
      torch.save(best_state_dict, save_path)
      pbar.write(f"Step {step + 1}, best model saved. (accuracy={best_accuracy:.4f})")

  pbar.close()


if __name__ == "__main__":
  main(**parse_args())


[Info]: Use cuda now!
